<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation file
path_to_data = '/content/fra.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs = pairs[1000:20000]


In [40]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [41]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:

english_sentences = [clean_sentence(pair[0]) for pair in pairs]
french_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

In [62]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

fra_pad_sentence1 = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
eng_pad_sentence1 = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# Reshape data
fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [68]:
input_sequence = Input(shape=(max_french_len,))
embedding = Embedding(input_dim=french_vocab, output_dim=128,)(input_sequence)

In [ ]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense


reviews =[
          'Never coming back!',
          'horrible service',
          'rude waitress',
          'cold food',
          'horrible food!',
          'awesome',
          'awesome services!',
          'rocks',
          'poor work',
          'couldn\'t have done better'
]
#Define labels
labels = array([1,1,1,1,1,0,0,0,0,0])

Vocab_size = 50
encoded_reviews = [one_hot(d,Vocab_size) for d in reviews]
print(f'encoded reviews: {encoded_reviews}')


max_length = 4
padded_reviews = pad_sequences(encoded_reviews,maxlen=max_length,padding='post')
print(padded_reviews)



model = Sequential()
embedding_layer = Embedding(input_dim=Vocab_size,output_dim=8,input_length=max_length)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
print(model.summary())

model.fit(padded_reviews,labels,epochs=100,verbose=0)




In [ ]:
input_data = np.array([[1,2,4,5]])
pred = model.predict(input_data)

print(input_data)
print(pred)
print(padded_reviews)

In [69]:
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)


In [ ]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

In [ ]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=30, epochs=100)


In [73]:
enc_dec_model.save('my_model.h5')

In [ ]:

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 14
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(french_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(, eng_text_tokenizer))

In [ ]:

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 14
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(french_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(, eng_text_tokenizer))

In [ ]:

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 14
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(french_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(fra_pad_sentence[index:index+1])[0], eng_text_tokenizer))
print("rfrif")
print(fra_pad_sentence[index:index+1])






In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')
index = 15


print(fra_text_tokenized)

print(fra_text_tokenizer.word_index)
print(fra_text_tokenized)


# fra_pad_sentence = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
# eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# fra_pad_sentence1 = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
# eng_pad_sentence1 = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# # Reshape data
# fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
# eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)


print(fra_pad_sentence[0])



In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')


print(fra_text_tokenized)

print(fra_text_tokenizer.word_index)
print(fra_text_tokenized)


# fra_pad_sentence = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
# eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# fra_pad_sentence1 = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
# eng_pad_sentence1 = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# # Reshape data
# fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
# eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)


In [ ]:
from numpy.core.fromnumeric import shape
fra_text = pad_sequences([[1,4,2404]], max_french_len, padding = "post")

fra_text = fra_text.reshape(*fra_text.shape, 1)
print(fra_text)
index = 30
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(french_sentences[index]))
print('The predicted sentence is :')
hhh,jjjjj=enc_dec_model.predict(fra_pad_sentence[index:index+1])[0], eng_text_tokenizer



print(np.argmax(hhh[1]))

print(eng_text_tokenizer.word_index)